# CSCE 470 :: Information Storage and Retrieval :: Texas A&M University :: Fall 2018


# Homework 3 (and 4):  Recommenders

### 100 points [10% of your final grade; that's double!]

### Due: November 8, 2018

*Goals of this homework:* Put your knowledge of recommenders to work. 

*Submission Instructions (Google Classroom):* To submit your homework, rename this notebook as  `lastname_firstinitial_hw#.ipynb`. For example, my homework submission would be: `caverlee_j_hw3.ipynb`. Submit this notebook via **Google Classroom**. Your IPython notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so).

# Part 0: Movielens Data

For this first part, we're going to use part of the Movielens 100k dataset. Prior to the Netflix Prize, the Movielens data was **the** most important collection of movie ratings.

First off, we need to load the data (including u.user, u.item, and ua.base). Here, we provide you with some helper code to load the data using [Pandas](http://pandas.pydata.org/). Pandas is a nice package for Python data analytics.

You may need to install pandas doing something like:

`conda install --name cs470 pandas`

In [1]:
import pandas as pd

# Load the user data
users_df = pd.read_csv('u.user', sep='|', names=['UserId', 'Age', 'Gender', 'Occupation', 'ZipCode'])

# Load the movies data: we will only use movie id and title for this homework
movies_df = pd.read_csv('u.item', sep='|', names=['MovieId', 'Title'], usecols=range(2), encoding = "ISO-8859-1")

# Load the ratings data: ignore the timestamps
ratings_df = pd.read_csv('ua.base', sep='\t', names=['UserId', 'MovieId', 'Rating'],usecols=range(3))

# Working on three different data frames is a pain
# Let us create a single dataset by "joining" these three data frames
movie_ratings_df = pd.merge(movies_df, ratings_df)
movielens_df = pd.merge(movie_ratings_df, users_df)

movielens_df.head()

,MovieId,Title,UserId,Rating,Age,Gender,Occupation,ZipCode
0,1,Toy Story (1995),1,5,24,M,technician,85711
1,2,GoldenEye (1995),1,3,24,M,technician,85711
2,3,Four Rooms (1995),1,4,24,M,technician,85711
3,4,Get Shorty (1995),1,3,24,M,technician,85711
4,5,Copycat (1995),1,3,24,M,technician,85711


# Part 1. Let's find similar users [20 points]

Before we get to the actual task of building our recommender, let's familiarize ourselves with the Movielens data.

Pandas is really nice, since it let's us do simple aggregates. For example, we can find a specific user and take a look at that user's ratings. For example, for the user with user ID = 363, we have:

In [2]:
gb = movielens_df.groupby('UserId')
User363 = gb.get_group(363)
#the information for the user
User363[:1][["UserId", "Age", "Gender","Occupation", "ZipCode"]]

,UserId,Age,Gender,Occupation,ZipCode
23594,363,20,M,student,87501


In [3]:
# And then we can see his first 10 ratings:
User363[['Title', 'Rating']][:10]

,Title,Rating
23594,Toy Story (1995),2
23595,GoldenEye (1995),4
23596,Get Shorty (1995),5
23597,Copycat (1995),1
23598,Twelve Monkeys (1995),3
23599,Babe (1995),5
23600,Dead Man Walking (1995),3
23601,Seven (Se7en) (1995),5
23602,"Usual Suspects, The (1995)",5
23603,From Dusk Till Dawn (1996),4


Balderdash! Everyone agrees that Toy Story should be rated 5! Oh well, there's no accounting for taste.

Moving on, let's try our hand at finding similar users to this base user (user ID = 363). In each of the following, **find the top-10 most similar users** to this base user. You should use all of the user's ratings, not just the top-10 like we showed above. We're going to try different similarity methods and see what differences arise.

You should implement each of these similar methods yourself! 

###     Top-10 Most Similar Users Using
#### Jaccard

In [4]:
import numpy as np
#setup ratings matrix

def getRatingMatrix(ratingsDf):
    matrix = np.zeros([ratingsDf['UserId'].max()+1, ratingsDf['MovieId'].max()+1])
    print(ratingsDf['UserId'].max()+1, ratingsDf['MovieId'].max()+1)
    gb = ratingsDf.groupby('UserId')
    for userId, group in gb:
        for mindex, mRow in group.iterrows():
            movieId = mRow['MovieId']
            matrix[userId][movieId] = int(mRow['Rating'])
    return matrix

In [5]:
ratingMatrix= getRatingMatrix(movielens_df)
ratingMatrixT = ratingMatrix.transpose()

944 1683


In [6]:
def jaccard(user1Id, user2Id, matrix):
    u1Vec = np.where(matrix[user1Id] > 0, 1, 0)
    u2Vec = np.where(matrix[user2Id] > 0, 1, 0)
    
    intersection = (np.dot(u1Vec, u2Vec))
    A = np.sum(u1Vec)
    B = np.sum(u2Vec)
    return intersection / (A + B - intersection) 

users_df_copy = users_df
jaccard_df = users_df_copy.apply(lambda row: pd.Series(
    {
        'UserId': row['UserId'],
        'jaccard': jaccard (363, row['UserId'], ratingMatrix)
    }),axis=1)
jaccard_df.sort_values('jaccard', ascending = False)[1:11]

,UserId,jaccard
275,276.0,0.392427
300,301.0,0.360577
895,896.0,0.357588
550,551.0,0.352814
915,916.0,0.351111
434,435.0,0.350806
292,293.0,0.349901
863,864.0,0.344828
91,92.0,0.344554
416,417.0,0.344262


####     Cosine

In [8]:
import math

def normalize(vec):
    magnitude = np.linalg.norm(vec)
    return vec / magnitude

def magnitude(vec):
    return np.linalg.norm(vec)

def cosine(user1Id, user2Id, matrix):
    u1Vec = matrix[user1Id]
    ANorm = magnitude(u1Vec)
    u2Vec = matrix[user2Id]
    BNorm = magnitude(u2Vec)
    dot = np.dot(u1Vec, u2Vec)
    return (dot / (ANorm*BNorm))
    
users_df_copy = users_df
users_df_copy = users_df_copy[users_df_copy.UserId != 363]

cosine_df = users_df_copy.apply(lambda row: pd.Series(
    {
        'UserId': row['UserId'],
        'cosine': cosine(363,row['UserId'], ratingMatrix)
    }),axis=1)
cosine_df.sort_values('cosine', ascending = False)[0:10]

,UserId,cosine
275,276.0,0.603857
863,864.0,0.530819
434,435.0,0.529115
302,303.0,0.522762
428,429.0,0.522503
895,896.0,0.518806
91,92.0,0.512386
681,682.0,0.511880
496,497.0,0.510762
221,222.0,0.510253


#### Pearson

In [9]:
#setup the intersection we'll use in the next part
#this seems a little more complicated than it needs to be, but its really slow otherwise
def intersection(user1Id, user2Id, matrix):
    #convert arrays to 1 or 0
    u1Vec = np.where(matrix[user1Id] > 0, 1, 0)
    u2Vec = np.where(matrix[user2Id] > 0, 1, 0)
    
    #find which elements both have 1's
    intersect = np.logical_and(u1Vec, u2Vec)
    
    #find all values that were 0 in the intersect and make them 0 
    u1Intersect = intersect * matrix[user1Id]
    u2Intersect = intersect * matrix[user2Id]
    
    #remove all the zeros
    return (np.extract(u1Intersect > 0, u1Intersect), np.extract(u2Intersect > 0, u2Intersect))

In [10]:
import math
import time

def pearson(user1Id, user2Id, matrix):
    (u1Vec, u2Vec) = intersection(user1Id, user2Id, matrix)
    u1Mean = np.mean(np.extract(matrix[user1Id] > 0, matrix[user1Id]))
    u2Mean = np.mean(np.extract(matrix[user2Id] > 0, matrix[user2Id]))
    u1Diff = (u1Vec - u1Mean)
    u2Diff = (u2Vec - u2Mean)
    top = (u1Diff * u2Diff).sum()
    bottom = math.sqrt((u1Diff*u1Diff).sum()*(u2Diff*u2Diff).sum())
    return top / bottom
    
users_df_copy = users_df
users_df_copy = users_df_copy[users_df_copy.UserId != 363]

pearson_df = users_df_copy.apply(lambda row: pd.Series(
    {
        'UserId': row['UserId'],
        'pearson': pearson (363, row['UserId'], ratingMatrix)
    }),axis=1)
pearson_df.sort_values('pearson', ascending = False)[0:10]


C:\Users\12r45\Anaconda3\envs\cs470\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


,UserId,pearson
219,220.0,1.000000
241,242.0,1.000000
684,685.0,1.000000
731,732.0,1.000000
154,155.0,1.000000
661,662.0,1.000000
281,282.0,1.000000
139,140.0,1.000000
340,341.0,0.992129
735,736.0,0.959680


### What are the differences among these three similarity methods? Which one do you prefer, why?

 Jaccard does not care about the rating that the user gave, only that it is included in the set of movies they rated, This causes it to maybe not be the best because it doesent take into account whether a user actually liked the movie. Cosine is better because it takes into account the rating, but it counts all of the movies that a user hasent rated but the other has as a 0, this may not be an accurate representation of a users score for that movie since they may have just not seen it yet. Pearson I think is the best because it takes into account ratings, and doesn't assume as much as cosine, because it only looks at movies that both have rated.

# Part 2: User-User Collaborative Filtering: Similarity-Based Ratings Prediction [20 points]

Now let's estimate the rating of UserID 363 for the movie "Dances with Wolves (1990)" (MovieId 97) based on the similar users. Find the 10 nearest (most similiar by using Pearson) users who rated the movie "Dances with Wolves (1990)" and try different aggregate functions. Recall, there are many different ways to aggregate the ratings of the nearest neighbors. We'll try three popular methods:

### Method 1: Average. 
The first is to simply average the ratings of the nearest neighbors:
$r_{c,s} = \frac{1}{N}\sum_{c'\in \hat{C}}r_{c',s}$

In [11]:
def getClosestkSimilarUsers(movieId, userId, similarityFunc, k, ratingsDF, matrix):
    #if(not (ratingsDF['MovieId'] == movieId).any()):
    
    movieRatings = ratingsDF.groupby('MovieId').get_group(movieId)
    movieRatings = movieRatings[movieRatings.UserId != userId]
    usersWithSimilarity = movieRatings.apply(lambda row: pd.Series(
        {
            'UserId': row['UserId'],
            'similarity': similarityFunc (userId, row['UserId'], matrix)
        }),axis=1)
    closestk = usersWithSimilarity.sort_values('similarity', ascending = False)[0:k]
    print(closestk)
    return pd.merge(movieRatings, closestk, how='inner', on=['UserId'])

print('predicted score = ', getClosestkSimilarUsers(97,363, pearson, 10, movie_ratings_df, ratingMatrix)['Rating'].mean())

       UserId  similarity
12462   187.0    0.550906
12652   889.0    0.460548
12429     1.0    0.449352
12539   453.0    0.449171
12472   226.0    0.437098
12583   615.0    0.423112
12543   457.0    0.421464
12434    24.0    0.409530
12627   781.0    0.406276
12482   263.0    0.399454
predicted score =  3.6


In [12]:
def getClosestkSimilarUsers(movieId, userId, similarityFunc, k, matrix, matrixT):
    usersWhoSawMovie = np.where(matrixT[movieId] > 0, 1, 0)
    userIndexes = np.fromfunction(lambda i,j: j, (1,len(usersWhoSawMovie)), dtype=int)[0]
    usersWhoSawMovieIndexes = (usersWhoSawMovie * userIndexes)
    usersWhoSawMovieIndexesFiltered = np.extract(usersWhoSawMovieIndexes > 0, usersWhoSawMovieIndexes)
    
    usersWithSim = (np.array(list(
        map(lambda item2: 
            (item2, similarityFunc (userId, item2, matrix), matrix[item2][movieId]),
             usersWhoSawMovieIndexesFiltered))))
    usersWithSim = np.nan_to_num(usersWithSim,0)
    sortedTopK = usersWithSim[usersWithSim[:,1].argsort()[::-1]][:k]
    return pd.DataFrame(data = sortedTopK, columns= ['UserId', 'similarity', 'Rating'])

getClosestkSimilarUsers(97,363, pearson, 10, ratingMatrix, ratingMatrixT)

,UserId,similarity,Rating
0,187.0,0.550906,3.0
1,889.0,0.460548,3.0
2,1.0,0.449352,3.0
3,453.0,0.449171,3.0
4,226.0,0.437098,3.0
5,615.0,0.423112,4.0
6,457.0,0.421464,5.0
7,24.0,0.409530,4.0
8,781.0,0.406276,4.0
9,263.0,0.399454,4.0


### Method 2: Weighted Average 1. 
The second is to take a weighted average, where we weight more "similar" neighbors higher: $r_{c,s} = k\sum_{c'\in \hat{C}}sim(c, c')\times r_{c',s}$

Choose a reasonable k so that r_{c,s} is between 1 to 5

In [13]:
def weightedAverage(movieId, userId, similarityFunc, k, matrix, matrixT):
    closestk = getClosestkSimilarUsers(movieId,userId, similarityFunc, k, matrix, matrixT)
    k = 1/closestk['similarity'].sum()
    score = closestk.apply(lambda row: k * row['similarity'] * row['Rating'], axis=1).sum()
    return score

print('predicted score = ', weightedAverage(97, 363, pearson, 10, ratingMatrix, ratingMatrixT))

predicted score =  3.563047602289183


### Method 3: Weighted Average 2. 
An alternative weighted average is to weight the differences between their ratings and their average rating (in essence to reward movies that are above the mean): $r_{c,s} = \bar{r}_c + k\sum_{c'\in \hat{C}}sim(c, c')\times (r_{c',s} - \bar{r}_{c'})$

Choose a reasonable k so that r_{c,s} is between 1 to 5

In [14]:
def weightedAverage2(movieId, userId, similarityFunc, k, matrix,matrixT):
    closestk = getClosestkSimilarUsers(movieId,userId, similarityFunc, k, matrix, matrixT)
    k = 1/closestk['similarity'].sum()
    rMean = np.mean(np.extract(matrix[userId] > 0, matrix[userId]))
    def innerSum(row):
        rUserId = int(row['UserId'])
        rOtherMean = np.mean(np.extract(matrix[rUserId] > 0, matrix[rUserId]))
        return row['similarity'] * (row['Rating'] - rOtherMean)
    score = rMean + k * closestk.apply(innerSum, axis=1).sum()

    return score

print('predicted score = ', weightedAverage2(97, 363, pearson, 5, ratingMatrix, ratingMatrixT))

predicted score =  2.4033934503512975


# Part 3: Baseline Recommendation (Global) [20 points]

OK, so far we've built the basics of a user-user collaborative filtering approach; that is, we take a user, find similar users and then aggregate their ratings. 

An alternative approach is to consider just basic statistics of the movies and users themselves. This is the essence of the "baseline" recommender we discussed in class:

$b_{xi} = \mu + b_x + b_i$

where $b_{x,i}$ is the baseline estimate rating user x would give to item i, $\mu$ is the overall mean rating, $b_x$ is a user bias term, and $b_i$ is an item bias term.

For this part, let's once again estimate the rating of UserID 363 for the movie "Dances with Wolves (1990)" (MovieId 97), but this time using the baseline recommender.

In [15]:
def getBaseline(userId, itemId, matrix, matrixT, avgGlobal):
    #we ask them to give us the global because it takes a long time to calculate
    start = time.time()
    #avgGlobal = np.extract(matrix > 0, matrix).mean()
    avgUser = np.extract(matrix[userId] > 0, matrix[userId]).mean()
    avgMovie = np.extract(matrixT[itemId] > 0, matrixT[itemId]).mean()
    end = time.time()
    #print(end - start)
    #print(avgGlobal + (avgUser - avgGlobal) + (avgMovie - avgGlobal))
    return avgGlobal + (avgUser - avgGlobal) + (avgMovie - avgGlobal)

avgGlobal = np.extract(ratingMatrix > 0, ratingMatrix).mean()
print('baseline for user 363, item 97 = ', getBaseline(363, 97, ratingMatrix, ratingMatrixT, avgGlobal))
#avgGlobal = np.extract(trainingRatingMatrix > 0, trainingRatingMatrix).mean()
#getBaseline(405, 1582, trainingRatingMatrix, trainingRatingMatrixT, avgGlobal)

baseline for user 363, item 97 =  3.3047530007520427


# Part 4: Local + Global Recommendation (Baseline + Item-Item CF) [20 points]

Our final recommender combines the global baseline recommender with an item-item local recommender. 

$\hat{r}_{xi} = b_{xi} + \dfrac{\sum_{j \in N(i;x)}s_{ij} \cdot (r_{xj} - b_{xj})}{\sum_{j \in N(i;x)}s_{ij}} $

where 
* $\hat{r}_{xi}$ is our estimated rating for what user x would give to item i.
* $s_{ij}$ is the similarity of items i and j.
* $r_{xj}$ is the rating of user x on item j.
* $N(i;x)$ is the set of items similar to item i that were rated by x.

You will need to make some design choices here about what similarity measure to use and what threshold to determine what items belong in $N(i;x)$.

Now show us what this estimates the rating of UserID 363 for the movie "Dances with Wolves (1990)" (MovieId 97) to be:

In [16]:
def itemItemClosestK(userId, movieId, k, matrix, matrixT, similarityFunc):
    #get the movies the user has seen as an array of 1s and 0s
    moviesUserHasSeen = np.where(matrix[userId] > 0, 1, 0)
    #get an array of just indexes for the movies
    movieIndexes = np.fromfunction(lambda i,j: j, (1,len(matrix[userId])), dtype=int)[0]
    #change the 1s to be the index of the movie it represents, and filter 0s
    moviesUserHasSeenIndexes = (moviesUserHasSeen * movieIndexes)
    moviesUserHasSeenFiltered = np.extract(moviesUserHasSeenIndexes > 0, moviesUserHasSeenIndexes)
    
    moviesWithSim = (np.array(list(map(lambda item2: (item2,similarityFunc (movieId, item2, matrixT)), moviesUserHasSeenFiltered))))
    sortedTopK = moviesWithSim[moviesWithSim[:,1].argsort()[::-1]][:k]
    
    return pd.DataFrame(data = sortedTopK, columns= ['MovieId', 'similarity'])

ratingMatrixT = ratingMatrix.transpose()
itemItemClosestK(363, 97, 10, ratingMatrix, ratingMatrixT, jaccard)


,MovieId,similarity
0,69.0,0.521614
1,423.0,0.504399
2,216.0,0.485207
3,172.0,0.481675
4,215.0,0.479592
5,196.0,0.476489
6,174.0,0.474820
7,82.0,0.470948
8,79.0,0.470430
9,204.0,0.466488


In [17]:
def localRating(userId, movieId, ratingsDf, matrix, matrixT, avgGlobal, k, similarityFunc):
    start = time.time()
    gb = ratingsDf.groupby('UserId')
    
    userRatings = gb.get_group(userId)
    
    top = 0
    bot = 0
    for mIndex, mRow in itemItemClosestK(userId, movieId, k, matrix, matrixT, similarityFunc).iterrows():
        sim = mRow['similarity']
        bot += sim
        rating = int(userRatings[userRatings.MovieId == mRow['MovieId']]['Rating'])
        base = getBaseline(userId, int(mRow['MovieId']), matrix, matrixT, avgGlobal)
        top += sim * (rating - base)
        
    end = time.time()
    #print('time', end-start)
    return (top / bot)

avgGlobal = np.extract(ratingMatrix > 0, ratingMatrix).mean()

print(getBaseline(363, 97, ratingMatrix, ratingMatrixT, avgGlobal) + localRating(363, 97, ratings_df, ratingMatrix, ratingMatrixT, avgGlobal, 1000, cosine))

3.348586264376746


# Part 5. Putting it all together! [20 points]

Finally, we're going to experiment with our different methods to see which one performs the best on our special test set of "hidden" ratings. We have three big "kinds" of recommenders:

* User-User Collaborative Filtering
* Baseline Recommendation (Global)
* Local + Global Recommender


But within each, we have lots of design choices. For example, do we try Jaccard+Average or Jaccard+WeightedAverage1? Do we try Jaccard or Cosine or Pearson? What choice of k? Etc.

For this part, you should train your methods on a special train set (the base set, see below). Then report your results over the test set (see below). You should use RMSE as your metric of choice. Which method performs best? You will need to experiment with many different approaches, but ultimately, you should tell us the best 2 or 3 methods and report the RMSE you get.

In [18]:
train = pd.read_csv('ua.base', sep='\t', names=['UserId', 'MovieId', 'Rating'],usecols=range(3))
test = pd.read_csv('ua.test', sep='\t', names=['UserId', 'MovieId', 'Rating'],usecols=range(3))


In [19]:
trainingRatingMatrix= getRatingMatrix(train)
trainingRatingMatrixT = trainingRatingMatrix.transpose()

944 1683


In [20]:
import time
import math
def GetRMSE(testDf, predictionFunction):
    rmse = 0
    cnt = 0
    for index, row in testDf.iterrows():
        start = time.time()
        score = predictionFunction(row)
        if(math.isnan(score)):
            score = 0
        sqrDiff = math.pow(row['Rating'] - score, 2)
        rmse += sqrDiff
        cnt+=1
    return math.sqrt(rmse/cnt)


In [21]:
#baseline
avgGlobal = np.extract(trainingRatingMatrix > 0, trainingRatingMatrix).mean()
def baselinePrediction(row):
    return getBaseline(row['UserId'], row['MovieId'], trainingRatingMatrix, trainingRatingMatrixT, avgGlobal)
GetRMSE(test, baselinePrediction)

C:\Users\12r45\Anaconda3\envs\cs470\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: Mean of empty slice.
  
C:\Users\12r45\Anaconda3\envs\cs470\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0.9931349305268995

In [56]:
def GetRMSE(testDf, predictionFunction, k, simFunc, simFuncName):
    rmse = 0
    cnt = 0
    for index, row in testDf.iterrows():
        start = time.time()
        score = predictionFunction(row, k, simFunc)
        if(math.isnan(score)):
            score = 0
        sqrDiff = math.pow(row['Rating'] - score, 2)
        rmse += sqrDiff
        cnt+=1
        
    return {'k': k, 'similarityFunc': simFuncName, 'error' : math.sqrt(rmse/cnt)}

In [72]:
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

def ParallelRMSE(testDf, predictionFunction, predictionFunctionName, ks, simFuncs):
    allPairs = [ (k,name,func) for k in ks for name, func in simFuncs.items()]
    results = Parallel(n_jobs=num_cores)(delayed(GetRMSE)(testDf,predictionFunction,k,func, name) for (k,name,func) in allPairs)
    error_df = pd.DataFrame(data=results)
    error_df['predictionFunction'] = predictionFunctionName
    return error_df

In [63]:
#user user
def userUserPrediction(row, k, simFunc):
    #start = time.time()
    #if we havent observed any rankings for this guy return global avg
    if(len(train[train.MovieId == row['MovieId']]) == 0):
        return avgGlobal
    score = weightedAverage(row['MovieId'], row['UserId'], simFunc, k, trainingRatingMatrix, trainingRatingMatrixT)
    #end = time.time()
    #print(end - start)
    return score

In [80]:
ks = [1,3,5,10,15,25]
similarityFuncs = {'jaccard' : jaccard,'cosine' : cosine,'pearson' : pearson}
ParallelRMSE(test, userUserPrediction, "userUserPrediction", ks, similarityFuncs)

,error,k,similarityFunc,predictionFunction
0,1.384299,1,jaccard,userUserPrediction
1,1.364430,1,cosine,userUserPrediction
2,1.422496,1,pearson,userUserPrediction
3,1.150847,3,jaccard,userUserPrediction
4,1.134679,3,cosine,userUserPrediction
5,4.722770,3,pearson,userUserPrediction
6,1.092170,5,jaccard,userUserPrediction
7,1.083332,5,cosine,userUserPrediction
8,1.166313,5,pearson,userUserPrediction
9,1.053130,10,jaccard,userUserPrediction


In [75]:
#user user2
def userUserPrediction2(row, k, simFunc):
    #if we havent observed any rankings for this guy return global avg
    if(len(train[train.MovieId == row['MovieId']]) == 0):
        return avgGlobal
    return weightedAverage2(row['MovieId'], row['UserId'], simFunc, k,trainingRatingMatrix, trainingRatingMatrixT)

In [81]:
ParallelRMSE(test, userUserPrediction2, "userUserPrediction2", ks, similarityFuncs)

,error,k,similarityFunc,predictionFunction
0,1.288981,1,jaccard,userUserPrediction2
1,1.274146,1,cosine,userUserPrediction2
2,1.347615,1,pearson,userUserPrediction2
3,1.077484,3,jaccard,userUserPrediction2
4,1.061590,3,cosine,userUserPrediction2
5,9.064044,3,pearson,userUserPrediction2
6,1.024510,5,jaccard,userUserPrediction2
7,1.014063,5,cosine,userUserPrediction2
8,1.071685,5,pearson,userUserPrediction2
9,0.985823,10,jaccard,userUserPrediction2


In [78]:
#local + global
#print(getBaseline(363, 97, ratingMatrix, ratingMatrixT, avgGlobal) + localRating(363, 97, ratings_df, ratingMatrix, ratingMatrixT, avgGlobal))
def localPlusGlobalPrediction(row, k, simFunc):
    base = getBaseline(row['UserId'], row['MovieId'], trainingRatingMatrix, trainingRatingMatrixT, avgGlobal)
    local = localRating(row['UserId'], row['MovieId'], train, trainingRatingMatrix, trainingRatingMatrixT, avgGlobal, k, simFunc)
    return local + base

"for i in ks:\n    k = i\n    for name, func in similarityFuncs.items():\n        simFunc = func\n        start = time.time()\n        error = GetRMSE(test[:1000], localPlusGlobalPrediction)\n        error_df = error_df.append(\n            {\n                'PredictionFunction':'Local Plus Global',\n                'k': k, \n                'similarityFunc': name,\n                'error' : error\n            }, ignore_index=True)\n        end = time.time()\n        print(end - start)\n        "

In [82]:
ParallelRMSE(test, localPlusGlobalPrediction, "localPlusGlobalPrediction", ks, similarityFuncs)

,error,k,similarityFunc,predictionFunction
0,1.216734,1,jaccard,localPlusGlobalPrediction
1,1.206042,1,cosine,localPlusGlobalPrediction
2,2.107633,1,pearson,localPlusGlobalPrediction
3,1.013133,3,jaccard,localPlusGlobalPrediction
4,1.010494,3,cosine,localPlusGlobalPrediction
5,2.004754,3,pearson,localPlusGlobalPrediction
6,0.973302,5,jaccard,localPlusGlobalPrediction
7,0.974181,5,cosine,localPlusGlobalPrediction
8,1.984047,5,pearson,localPlusGlobalPrediction
9,0.947908,10,jaccard,localPlusGlobalPrediction


In [51]:
error_df

,k,similarityFunc,error
0,1.0,jaccard,1.384299
1,1.0,cosine,1.364430
2,1.0,pearson,2.687439
3,3.0,jaccard,1.150847
4,3.0,cosine,1.134679
5,3.0,pearson,5.071950
6,5.0,jaccard,1.092170
7,5.0,cosine,1.083332
8,5.0,pearson,1.982443
9,10.0,jaccard,1.053130


*provide your best 2 or 3 methods, their RMSE, plus some discussion of why they did the best*

### BONUS: 
Can you do better? Find a way to improve the results!

In [ ]:
# your code here